In [1]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_141c90d309b2415da230d859c23fce86 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='hzXembrCjQSB2jSQ_kICFB2_T4vastDAbVZrmuW4VofA',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_141c90d309b2415da230d859c23fce86.get_object(Bucket='week2-donotdelete-pr-t32uj9grxbaezt',Key='Churn_Modelling.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df = pd.read_csv(body)
df.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## ETL AND PREPROCESSING THE DATA

In [2]:
df= df.drop(['RowNumber', 'CustomerId', 'Surname'], axis =1)

In [3]:
#to calculate the null values in the data
df.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

###### To encode the categorical data using label encoder

In [10]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
df['Geography']=le.fit_transform(df['Geography'])
df['Gender'] = le.fit_transform(df['Gender'])

###### Selecting k best features to make the model robust and loose redundant data

In [12]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
x=df.drop('Exited',axis=1)
y=df['Exited']
#feature engineering to select importent features
bestfeatures = SelectKBest(score_func=chi2, k=7)
fit = bestfeatures.fit(x,y)
df_scores = pd.DataFrame(fit.scores_)
df_columns = pd.DataFrame(df.columns)
#concat two dataframes for better visualization 
feature_scores = pd.concat([df_columns,df_scores],axis=1)
feature_scores.columns = ['Column','Score']  #naming the dataframe columns
print(feature_scores.nlargest(7,'Score'))  #print 7 best features

            Column         Score
5          Balance  7.151303e+06
9  EstimatedSalary  4.835088e+04
3              Age  2.300417e+03
8   IsActiveMember  1.181994e+02
0      CreditScore  1.054035e+02
2           Gender  5.153993e+01
1        Geography  1.185325e+01


###### Standardising the data

In [17]:
from sklearn.preprocessing import StandardScaler
continuous_data=df.drop(['Geography','Gender','Exited'], axis=1)
sc=StandardScaler()
x=sc.fit_transform(continuous_data)
x=pd.DataFrame(x,columns=continuous_data.columns)
x.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,-0.326221,0.293517,-1.041760,-1.225848,-0.911583,0.646092,0.970243,0.021886
1,-0.440036,0.198164,-1.387538,0.117350,-0.911583,-1.547768,0.970243,0.216534
2,-1.536794,0.293517,1.032908,1.333053,2.527057,0.646092,-1.030670,0.240687
3,0.501521,0.007457,-1.387538,-1.225848,0.807737,-1.547768,-1.030670,-0.108918
4,2.063884,0.388871,-1.041760,0.785728,-0.911583,0.646092,0.970243,-0.365276


###### Creating final data for model building

In [19]:
x = pd.concat([x,df['Geography'],df['Gender']], axis=1)
y = df['Exited']

###### Dividing into train and test data

In [20]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [22]:
print('The length of train data is {} and test data is {} for x'.format(len((x_train)),len(x_test)))
print('The length of train data is {} and test data is {} for y'.format(len((y_train)),len(y_test)))

The length of train data is 8000 and test data is 2000 for x
The length of train data is 8000 and test data is 2000 for y
